# Architectural factor binding in YY1-AID cells
Goal: determine whether Rad21 binding patterns change after depletion of YY1 (in the YY1-AID7 cell line, 4hr auxin treatment, asynchronous cells

In [ ]:
library(DiffBind);
library(tidyverse);
library(rtracklayer);

In [ ]:
config <- 'config_diffbind_q05.csv'

In [ ]:
# read in peak info based on config file
samples <- read.csv(config)
samples <- samples %>% filter(Factor=="RAD21")
samples

In [ ]:
db <- dba(sampleSheet=samples, peakCaller='narrow',
    config=data.frame(AnalysisMethod=DBA_DESEQ2,th=0.05,
                      DataType=DBA_DATA_FRAME, RunParallel=TRUE, 
                      minQCth=20, fragmentSize=250, 
                      bCorPlot=FALSE, reportInit="DBA", 
                      bUsePval=FALSE, design=TRUE,
                      doBlacklist=TRUE,
                      doGreylist=FALSE))

In [ ]:
# remove blacklist regions
db <- dba.blacklist(db, blacklist=DBA_BLACKLIST_MM9)

In [ ]:
# count reads; default summit = 200bp (ie 400bp interval)
db <- dba.count(db, summits=200, minOverlap=1, bParallel=FALSE)

In [ ]:
# save peak counts
for (i in 1:nrow(samples)){
    samplename <- db$samples$SampleID[i]
    db$peaks[[i]]['sample'] <- samplename
}
y <- do.call(rbind, db$peaks)
write.csv(y, file='Y7ASYNC_RAD21_peak_counts_q05.csv', row.names = FALSE)
write.csv(db$called, file='Y7ASYNC_RAD21_peak_occupancy_q05.csv', row.names = FALSE)